In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 0. 데이터 로드

In [ ]:
cd /content/drive/MyDrive/2021/날씨 공모전/code

/content/drive/MyDrive/2021/날씨 공모전/code


## 0-1. 날씨

#### 0-1-1. 지역별

In [ ]:
weather = pd.read_csv('../data/weather_final.csv', encoding='cp949')
weather['날짜'] = weather['날짜'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))

weather.head()

,날짜,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부
0,2018-01-01,105,1.3,5.7,-2.1,0.0,0.0,3.7,강릉,20.066667,13.400000,25.4,57.9,1023.0,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0
1,2018-01-01,112,-0.3,2.7,-2.7,0.0,0.0,1.6,인천,37.518681,18.641758,67.2,53.8,1020.3,2018,1,1,1,0,1월1일,1,0,3,9.992379,-42.924922,0,0
2,2018-01-01,119,-1.7,4.7,-6.9,0.0,0.0,1.0,수원,42.782895,21.375000,84.7,52.7,1025.1,2018,1,1,1,0,1월1일,1,0,3,10.136402,-45.791893,0,0
3,2018-01-01,136,-1.0,4.7,-6.5,0.0,0.0,2.2,안동,39.750000,28.583333,56.4,58.3,1010.6,2018,1,1,1,0,1월1일,1,0,3,8.356692,-44.375704,0,0
4,2018-01-01,152,2.1,6.2,-0.4,0.0,0.0,3.3,울산,38.548193,15.433735,42.8,58.3,1017.1,2018,1,1,1,0,1월1일,1,0,3,8.861429,-38.064334,0,0


#### 0-1-2. 날씨지수

In [ ]:
'''
weather_idx = pd.read_csv('../data/data_with_weather.csv', encoding='cp949')
weather_idx['date'] = weather_idx['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
weather_idx.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa
0,2018-01-01,F,20,식품,가공란,37,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
1,2018-01-01,F,30,식품,가공란,16,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
2,2018-01-01,F,40,식품,가공란,9,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
3,2018-01-01,F,50,식품,가공란,3,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
4,2018-01-01,M,20,식품,가공란,13,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385


## 0-2. 구매이력 및 검색량

In [ ]:
'''
trend_weather = pd.read_csv('../data/trend_with_weather.csv', encoding='cp949')
trend_weather['date'] = trend_weather['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
trend_weather.head()

,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보
0,2018-01-01,F,20,식품,가공란,37.0,0.0,10.69387,4.09137,0.00000,0.00000,0.04440
1,2018-01-01,F,30,식품,가공란,16.0,0.0,12.42969,6.35294,0.00000,0.51282,0.42417
2,2018-01-01,F,40,식품,가공란,9.0,0.0,13.02249,4.88037,0.00000,0.40650,0.00000
3,2018-01-01,F,50,식품,가공란,3.0,0.0,11.69080,4.53350,1.21951,0.90909,0.00000
4,2018-01-01,M,20,식품,가공란,13.0,0.0,11.23032,3.61296,0.38910,2.22222,0.04060


In [ ]:
# trend_weather.shape

(2568668, 12)

## 0-3. 공휴일

In [ ]:
'''
calender = weather[['날짜', '분기', '요일', '공휴일명', '공휴일여부', '주말여부', '계절']]
calender.head()

,날짜,분기,요일,공휴일명,공휴일여부,주말여부,계절
0,2018-01-01,1,0,1월1일,1,0,3
1,2018-01-01,1,0,1월1일,1,0,3
2,2018-01-01,1,0,1월1일,1,0,3
3,2018-01-01,1,0,1월1일,1,0,3
4,2018-01-01,1,0,1월1일,1,0,3


## 0-4. 클러스터 정보

In [ ]:
food_cluster = pd.read_csv('../data/food_tsclustering_norm.csv', encoding='cp949')[['sm_cat', 'cluster']]
food_cluster.head()

,sm_cat,cluster
0,가공란,3
1,가자미,3
2,갈비/찜/바비큐용 돈육,3
3,갈비용 우육,3
4,갈치,1


# 1. 데이터 preprocessing

In [ ]:
'''
data = trend_weather.merge(weather_idx.drop(['qty'], axis=1), how='left', on=['date', 'sex', 'age', 'big_cat', 'sm_cat'])
data = data.merge(calender.groupby(['날짜']).mean().reset_index(), left_on='date', right_on='날짜')
data.shape
'''

(2568668, 30)

In [ ]:
data = pd.read_csv('../data/final_0606.csv', encoding='cp949').iloc[:,1:]
data['date'] = data['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
print(data.shape)
data.head()

(2568668, 30)


,date,sex,age,big_cat,sm_cat,qty,cnt,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,분기,요일,공휴일명,공휴일여부,주말여부,계절
0,2018-01-01,F,20,식품,가공란,37.0,0.480964,0.0,10.69387,4.09137,0.00000,0.00000,0.04440,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
1,2018-01-01,F,30,식품,가공란,16.0,0.480964,0.0,12.42969,6.35294,0.00000,0.51282,0.42417,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
2,2018-01-01,F,40,식품,가공란,9.0,0.480964,0.0,13.02249,4.88037,0.00000,0.40650,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
3,2018-01-01,F,50,식품,가공란,3.0,0.480964,0.0,11.69080,4.53350,1.21951,0.90909,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
4,2018-01-01,M,20,식품,가공란,13.0,0.480964,0.0,11.23032,3.61296,0.38910,2.22222,0.04060,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3


In [ ]:
def region_weather(지역):
    weather_true = weather.drop(['연','월', '일', '분기', '요일', '공휴일명', '공휴일여부', '주말여부', '계절'], axis=1) # weather_final
    weather1 = pd.pivot_table(weather_true[weather_true['지역'].isin(지역)], index='날짜', columns='지역').reset_index(drop=True)
    weather1.columns = [l+"_"+w for l,w in zip(weather1.columns.get_level_values(1), weather1.columns.get_level_values(0))]
    weather1['날짜'] = weather['날짜'].unique()
    weather1['연'] = weather['날짜'].apply(lambda x : x.year)
    weather1['월'] = weather['날짜'].apply(lambda x : x.month)
    return weather1

In [ ]:
food = data[data.big_cat=='식품']
food = food.merge(food_cluster, on='sm_cat')
print("food_full's shape : {}".format(food.shape))

food0 = food[food.cluster==0].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)
food1 = food[food.cluster==1].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)
food2 = food[food.cluster==2].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)
food3 = food[food.cluster==3].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)
food4 = food[food.cluster==4].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)

food0 = food0.merge(region_weather(['서울', '부산', '제주']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)
food1 = food1.merge(region_weather(['서울', '부산', '수원']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)
food2 = food2.merge(region_weather(['서울', '부산', '대구']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)
food3 = food3.merge(region_weather(['서울', '부산', '안동']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)
food4 = food4.merge(region_weather(['서울', '천안', '안동']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)

print("food cluster0's shape : {}".format(food0.shape))
print("food cluster1's shape : {}".format(food1.shape))
print("food cluster2's shape : {}".format(food2.shape))
print("food cluster3's shape : {}".format(food3.shape))
print("food cluster4's shape : {}".format(food4.shape))

food_full's shape : (1496505, 31)
food cluster0's shape : (69528, 78)
food cluster1's shape : (235006, 78)
food cluster2's shape : (389768, 78)
food cluster3's shape : (680001, 78)
food cluster4's shape : (122202, 78)


In [ ]:
food0.columns

Index(['date', 'sex', 'age', 'sm_cat', 'qty', 'cnt', 'ratio', '오늘날씨', '미세먼지',
       '오늘 비', '기상정보', '태풍정보', 'avg_ta', 'max_ta', 'min_ta', 'rn_day',
       'rn_hr1', 'avg_ws', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa',
       '분기', '요일', '공휴일여부', '주말여부', '계절', '부산_1시간최대강수량', '서울_1시간최대강수량',
       '제주_1시간최대강수량', '부산_1시간최대습도', '서울_1시간최대습도', '제주_1시간최대습도', '부산_PM10',
       '서울_PM10', '제주_PM10', '부산_PM25', '서울_PM25', '제주_PM25', '부산_강수여부',
       '서울_강수여부', '제주_강수여부', '부산_열지수', '서울_열지수', '제주_열지수', '부산_일별강수량',
       '서울_일별강수량', '제주_일별강수량', '부산_일조시간합', '서울_일조시간합', '제주_일조시간합', '부산_지점번호',
       '서울_지점번호', '제주_지점번호', '부산_체감온도', '서울_체감온도', '제주_체감온도', '부산_최고기온',
       '서울_최고기온', '제주_최고기온', '부산_최고현지기압', '서울_최고현지기압', '제주_최고현지기압', '부산_최저기온',
       '서울_최저기온', '제주_최저기온', '부산_평균기온', '서울_평균기온', '제주_평균기온', '부산_평균풍속',
       '서울_평균풍속', '제주_평균풍속', '부산_폭염여부', '서울_폭염여부', '제주_폭염여부', '연', '월'],
      dtype='object')

In [ ]:
def get_dummies(data, var_list):
    # var_list = ['var1', 'var2']
    for var in var_list:
        if var=='sex':
            data[var] = pd.get_dummies(data[var]).iloc[:,:1] # F(1), M(0)
        else:
            data = pd.concat([data.drop(var, axis=1), pd.get_dummies(data[var], prefix=var, drop_first=True)], axis=1)
    return data

In [ ]:
food0 = get_dummies(food0, ['age', 'sex', 'sm_cat'])
food1 = get_dummies(food1, ['age', 'sex', 'sm_cat'])
food2 = get_dummies(food2, ['age', 'sex', 'sm_cat'])
food3 = get_dummies(food3, ['age', 'sex', 'sm_cat'])
food4 = get_dummies(food4, ['age', 'sex', 'sm_cat'])

In [ ]:
food0.head()

,date,sex,qty,cnt,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,분기,요일,공휴일여부,주말여부,계절,부산_1시간최대강수량,서울_1시간최대강수량,제주_1시간최대강수량,부산_1시간최대습도,서울_1시간최대습도,제주_1시간최대습도,부산_PM10,서울_PM10,제주_PM10,부산_PM25,서울_PM25,제주_PM25,부산_강수여부,서울_강수여부,...,제주_일조시간합,부산_지점번호,서울_지점번호,제주_지점번호,부산_체감온도,서울_체감온도,제주_체감온도,부산_최고기온,서울_최고기온,제주_최고기온,부산_최고현지기압,서울_최고현지기압,제주_최고현지기압,부산_최저기온,서울_최저기온,제주_최저기온,부산_평균기온,서울_평균기온,제주_평균기온,부산_평균풍속,서울_평균풍속,제주_평균풍속,부산_폭염여부,서울_폭염여부,제주_폭염여부,연,월,age_30,age_40,age_50,age_60,sm_cat_견과류 카카오닙스,sm_cat_바나나우유,sm_cat_에이드,sm_cat_유제품 음료,sm_cat_잡곡 씨드류,sm_cat_전통차,sm_cat_초코우유,sm_cat_해초류,sm_cat_허브차
0,2018-01-01,1,2.0,3.139177,5.17851,10.69387,4.09137,0.00000,0.00000,0.04440,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0.0,0.0,0.0,46.1,57.1,66.5,44.615079,42.307692,48.255319,19.728175,21.470696,21.765957,0,0,...,13.1,159,108,184,10.348688,9.649709,12.151456,7.4,3.8,8.8,1019.1,1018.1,1027.4,-1.1,-5.1,4.0,2.7,-1.3,6.4,2.7,1.4,3.5,0,0,0,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,1,7.0,3.139177,4.93892,12.42969,6.35294,0.00000,0.51282,0.42417,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0.0,0.0,0.0,46.1,57.1,66.5,44.615079,42.307692,48.255319,19.728175,21.470696,21.765957,0,0,...,13.1,159,108,184,10.348688,9.649709,12.151456,7.4,3.8,8.8,1019.1,1018.1,1027.4,-1.1,-5.1,4.0,2.7,-1.3,6.4,2.7,1.4,3.5,0,0,0,2018,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,1,4.0,3.139177,4.22489,13.02249,4.88037,0.00000,0.40650,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0.0,0.0,0.0,46.1,57.1,66.5,44.615079,42.307692,48.255319,19.728175,21.470696,21.765957,0,0,...,13.1,159,108,184,10.348688,9.649709,12.151456,7.4,3.8,8.8,1019.1,1018.1,1027.4,-1.1,-5.1,4.0,2.7,-1.3,6.4,2.7,1.4,3.5,0,0,0,2018,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,1,3.0,3.139177,4.61393,11.69080,4.53350,1.21951,0.90909,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0.0,0.0,0.0,46.1,57.1,66.5,44.615079,42.307692,48.255319,19.728175,21.470696,21.765957,0,0,...,13.1,159,108,184,10.348688,9.649709,12.151456,7.4,3.8,8.8,1019.1,1018.1,1027.4,-1.1,-5.1,4.0,2.7,-1.3,6.4,2.7,1.4,3.5,0,0,0,2018,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,1,0.0,0.000000,3.34728,13.92712,5.47887,0.00000,0.00000,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0.0,0.0,0.0,46.1,57.1,66.5,44.615079,42.307692,48.255319,19.728175,21.470696,21.765957,0,0,...,13.1,159,108,184,10.348688,9.649709,12.151456,7.4,3.8,8.8,1019.1,1018.1,1027.4,-1.1,-5.1,4.0,2.7,-1.3,6.4,2.7,1.4,3.5,0,0,0,2018,1,0,0,0,1,0,0,0,0,0,0,0,0,0


# 2. 모델링

In [ ]:
! pip install shap

     |████████████████████████████████| 358kB 14.8MB/s eta 0:00:01
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491628 sha256=807755a785ccba174afc580fb31114199e29078f68b64dc5cde9bc96d437afc7
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
import shap
import random
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor

## 2-1. baseline

### 2-1-1. food0

#### Train and Test split

In [ ]:
def sampling(data):
    random.seed(1234)
    # 랜덤으로 요일 선택
    sample_day = []; sample_idx = []
    count = 730//7 
    day_list = day_list = [1,2,3,4,5,6,7]
    sample_list = [random.choice(day_list) for i in range(count)]

    # 랜덤으로 선택된 요일의 날짜
    start = datetime.datetime(2018, 1, 1)
    for days in sample_list:
        sample_day.append(start + datetime.timedelta(days = days))
        start = start + datetime.timedelta(weeks = 1)

    # 샘플링
    for date in sample_day:
        sample_idx.extend(data[data['date']==date].index)
        
    all_idx = data.index.tolist()
    train_idx = list(set(all_idx) - set(sample_idx))
    train = data.iloc[train_idx].reset_index(drop=True)
    test = data.iloc[sample_idx].reset_index(drop=True)
    return train.drop('date', axis=1), test.drop('date', axis=1)

In [ ]:
train, test = sampling(food0)

In [ ]:
def train_xgboost(data):
    
    # train and test split 
    train, test = sampling(data)
    
    # standardization 
    std_scaler = StandardScaler()
    train_std = std_scaler.fit_transform(train); test_std = std_scaler.transform(test)
    
    X_train = train_std.iloc[:,]; y_train = train_std.iloc[:,]
    X_test = test_std.iloc[:]; y_test = test_std.iloc[:,]
    
    xgb_model = xgb.XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100) 
    xgb_model.fit(X_train, y_train)
    
    

In [ ]:
# ======== train and test split ========

train_0, test_0 = sampling(food0)
train_1, test_1 = sampling(food1)
train_2, test_2 = sampling(food2)
train_3, test_3 = sampling(food3)
train_4, test_4 = sampling(food4)

# ======== standardization ========

std_scaler = StandardScaler()
train_std_0 = std_scaler.fit_transform(train_0); test_std_0 = std_scaler.transform(test_0)

std_scaler = StandardScaler()
train_std_1 = std_scaler.fit_transform(train_1); test_std_1 = std_scaler.transform(test_1)

std_scaler = StandardScaler()
train_std_2 = std_scaler.fit_transform(train_2); test_std_2 = std_scaler.transform(test_2)

std_scaler = StandardScaler()
train_std_3 = std_scaler.fit_transform(train_3); test_std_3 = std_scaler.transform(test_3)

std_scaler = StandardScaler()
train_std_4 = std_scaler.fit_transform(train_4); test_std_4 = std_scaler.transform(test_4)

# ======== training =========



In [ ]:
x, y = np.array([train_std_0, test_std_0])
x

array([[ 0.98685151, -0.42154722, -0.37131284, ..., -0.32568501,
        -0.32675511, -0.34142928],
       [ 0.98685151, -0.2119591 , -0.37131284, ..., -0.32568501,
        -0.32675511, -0.34142928],
       [ 0.98685151, -0.33771197, -0.37131284, ..., -0.32568501,
        -0.32675511, -0.34142928],
       ...,
       [-1.01332368, -0.04428861, -0.37834619, ..., -0.32568501,
        -0.32675511,  2.9288642 ],
       [-1.01332368, -0.17004148, -0.37834619, ..., -0.32568501,
        -0.32675511,  2.9288642 ],
       [-1.01332368, -0.3796296 , -0.37834619, ..., -0.32568501,
        -0.32675511,  2.9288642 ]])

In [ ]:
print(train_0.shape, test_0.shape)
print(train_1.shape, test_1.shape)
print(train_2.shape, test_2.shape)
print(train_3.shape, test_3.shape)
print(train_4.shape, test_4.shape)

(59615, 88) (9913, 88)
(201492, 112) (33514, 112)
(334169, 134) (55599, 134)
(583010, 173) (96991, 173)
(104786, 95) (17416, 95)


#### Standardization

In [ ]:
std_scaler = StandardScaler()
train_std_0 = std_scaler.fit_transform(train_0); test_std_0 = std_scaler.transform(test_0)
train_std_1 = std_scaler.fit_transform(train_1); test_std_1 = std_scaler.transform(test_1)
train_std_2 = std_scaler.fit_transform(train_2); test_std_2 = std_scaler.transform(test_2)
train_std_3 = std_scaler.fit_transform(train_3); test_std_3 = std_scaler.transform(test_3)
train_std_4 = std_scaler.fit_transform(train_4); test_std_4 = std_scaler.transform(test_4)

#### X, y split

In [ ]:

train_0.columns.difference(['qty','age'])

Index(['PM10', 'PM25', 'age_30', 'age_40', 'age_50', 'age_60', 'avg_ta',
       'avg_ws', 'cnt', 'hm_max', 'max_pa', 'max_ta', 'min_ta', 'ratio',
       'rn_day', 'rn_hr1', 'sex', 'sm_cat_견과류 카카오닙스', 'sm_cat_바나나우유',
       'sm_cat_에이드', 'sm_cat_유제품 음료', 'sm_cat_잡곡 씨드류', 'sm_cat_전통차',
       'sm_cat_초코우유', 'sm_cat_해초류 ', 'sm_cat_허브차', 'sum_ss_hr', '계절', '공휴일여부',
       '기상정보', '미세먼지', '부산_1시간최대강수량', '부산_1시간최대습도', '부산_PM10', '부산_PM25',
       '부산_강수여부', '부산_열지수', '부산_일별강수량', '부산_일조시간합', '부산_지점번호', '부산_체감온도',
       '부산_최고기온', '부산_최고현지기압', '부산_최저기온', '부산_평균기온', '부산_평균풍속', '부산_폭염여부',
       '분기', '서울_1시간최대강수량', '서울_1시간최대습도', '서울_PM10', '서울_PM25', '서울_강수여부',
       '서울_열지수', '서울_일별강수량', '서울_일조시간합', '서울_지점번호', '서울_체감온도', '서울_최고기온',
       '서울_최고현지기압', '서울_최저기온', '서울_평균기온', '서울_평균풍속', '서울_폭염여부', '연', '오늘 비',
       '오늘날씨', '요일', '월', '제주_1시간최대강수량', '제주_1시간최대습도', '제주_PM10', '제주_PM25',
       '제주_강수여부', '제주_열지수', '제주_일별강수량', '제주_일조시간합', '제주_지점번호', '제주_체감온도',
       '제주_최고기온', '제주_최고현지기압', '제주_최

In [ ]:
train_Y_0, train_X_0 = train_num_0[['qty']], train_num_0[train_std_0.columns.difference(['qty','age'])]
test_Y_0, test_X_0 = test_num_0[['qty']], test_num_0[test_num_0.columns.difference(['qty','age'])]
train_Y_1, train_X_1 = train_num_1[['qty']], train_num_1[train_num_1.columns.difference(['qty','age'])]
test_Y_1, test_X_1 = test_num_1[['qty']], test_num_1[test_num_1.columns.difference(['qty','age'])]
train_Y_2, train_X_2 = train_num_2[['qty']], train_num_2[train_num_2.columns.difference(['qty','age'])]
test_Y_2, test_X_2 = test_num_2[['qty']], test_num_2[test_num_2.columns.difference(['qty','age'])]

NameError: ignored